In [3]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.stem import WordNetLemmatizer, SnowballStemmer, LancasterStemmer
from nltk.stem.porter import *
import nltk
from nltk.tag import pos_tag
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anirudhparameswaran/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
df = pd.read_csv('wiki_movie_plots_deduped.csv')
df = df[df['Origin/Ethnicity'] == 'Bollywood']
df = df[df['Release Year'] >= 2013].sample(200)
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
26867,2013,Table No. 21,Bollywood,Aditya Datt,"Paresh Rawal, Rajeev Khandelwal, Tena Desae",thriller/drama,https://en.wikipedia.org/wiki/Table_No._21,Vivaan (Rajeev Khandelwal) and Siya Agasthi (T...
27051,2017,Runningshaadi.com,Bollywood,Amit Roy,"Taapsee Pannu, Amit Sadh & Arsh Bajwa",comedy,https://en.wikipedia.org/wiki/Runningshaadi.com,The story revolves around three characters — R...
26985,2014,Manjunath,Bollywood,Sandeep A. Varma,"Divya Dutta, Seema Biswas, Asif Basra, Yashpal...",drama,https://en.wikipedia.org/wiki/Manjunath_(Film),In the hinterland of Uttar Pradesh 27-year-old...
26917,2013,Chennai Express,Bollywood,Rohit Shetty,"Shah Rukh Khan, Deepika Padukone[89]",comedy/action,https://en.wikipedia.org/wiki/Chennai_Express,Rahul Mithaiwala (Shah Rukh Khan) is an orphan...
26897,2013,Yamla Pagla Deewana 2,Bollywood,Sangeeth Sivan,"Dharmendra, Sunny Deol, Bobby Deol, Neha Sharm...",comedy/action,https://en.wikipedia.org/wiki/Yamla_Pagla_Deew...,"In Varanasi, Dharam Dhillon (Dharmendra) and h..."


In [ ]:
plots = df['Plot'].tolist()
titles = df['Title'].tolist()
genre = df['Genre'].tolist()
df['combined'] = 'Title: ' + df['Title'] + '\nGenre: ' + df['Genre'] + '\nPlot: ' + df['Plot']

# preprocessing
clean_re = r'[^a-zA-Z\s]'
wnl = WordNetLemmatizer()

input = df['combined'].tolist()
for plot in input:
    plot = re.sub(clean_re, '', plot)
    plot = plot.lower()


vectorizer_model = CountVectorizer(stop_words="english")
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(input)

2025-11-29 19:50:02,455 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 7/7 [00:02<00:00,  2.54it/s]
2025-11-29 19:50:07,455 - BERTopic - Embedding - Completed ✓
2025-11-29 19:50:07,455 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-29 19:50:07,688 - BERTopic - Dimensionality - Completed ✓
2025-11-29 19:50:07,689 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-29 19:50:07,727 - BERTopic - Cluster - Completed ✓
2025-11-29 19:50:07,735 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-29 19:50:07,817 - BERTopic - Representation - Completed ✓


In [12]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,96,-1_plot_title_genre_love,"[plot, title, genre, love, police, tells, sing...",[Title: Chhota Bheem And The Throne of Bali\nG...
1,0,60,0_father_love_day_family,"[father, love, day, family, tells, wedding, ti...",[Title: Mubarakan\nGenre: comedy/romance\nPlot...
2,1,44,1_khan_raja_police_rajveer,"[khan, raja, police, rajveer, kill, harleen, l...",[Title: Once Upon Ay Time In Mumbai Dobaara!\n...
